# Quick and dirty MOC

## part of a project to quantify transports across the Greenland-Scotland section by source and destination

- using output from Ocean Parcels, but just the initial particle properties
- random positioned particles with T,S and velocity components
- using these to diagnose transports by density and velocity bin

## technical preamble

In [ ]:
%matplotlib inline

In [ ]:


from pathlib import Path
import numpy as np
import xarray as xr
from datetime import datetime, timedelta
import seaborn as sns
import cmocean as co
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seawater as sw
np.warnings.filterwarnings('ignore')


## parameters

In [ ]:
# release time

trelease = np.datetime64('2015-12-16T12:00')

# Project path
project_path = Path.cwd() / '..' / '..' 
project_path = project_path.resolve()

# project_path = Path("C:/Users/sa05af/OneDrive - SAMS/Projects/parcels_quantitative_202004/")

# select section FS, IF, or GI

section = 'FS'

if section == 'FS':
    
    # Faroe-Shetland
    path_data_tracks = Path('data/processed/tracks/FaroeShetland/') 
    filename = 'test_salinity_FS.nc'
#     filenameb = 'tracks_parcels_FS_backwards_201512.nc'

    #velocity conversions on FS channel (61N)

    lat2metres = 60.0*1852.0
    lon2metres = 60.0*1852.0*np.cos(np.radians(61.0))

    sealandratio = 5591.0/20000.0
    sectionAreaEW = 5.7 * 1500.0 * sealandratio * lon2metres  # very approx. mean depth x length between slopes.
    sectionAreaNS = 1.87 * 1500.0 * sealandratio * lat2metres
    sectionArea = np.sqrt(sectionAreaEW**2 + sectionAreaNS**2)
    sectionAngle = np.arctan2(1.87 * lat2metres, 5.7 * lon2metres)
    
# elif section == 'IF':
    
#     # Iceland-Faroe
#     path_data_tracks = Path('data/processed/tracks/IcelandFaroe/') 
#     filename = 'tracks_parcels_IFR_forwards_201512.nc'
#     filenameb = 'tracks_parcels_IFR_backwards_201512.nc'

#     #velocity conversions on IFR (63.5N)

#     lat2metres = 60.0*1852.0
#     lon2metres = 60.0*1852.0*np.cos(np.radians(63.5))

#     sealandratio = 9308.0/20000.0
#     sectionAreaEW = 7.0 * 700.0 * sealandratio * lon2metres  # very approx. mean depth x length between slopes.
#     sectionAreaNS = 2.8 * 700.0 * sealandratio * lat2metres
    
# else:

#     # Greenland-Iceland
#     path_data_tracks = Path('data/processed/tracks/GreenlandIceland/') 
#     filename = 'tracks_parcels_GI_forwards_201512.nc'
#     filenameb = 'tracks_parcels_GI_backwards_201512.nc'

#     #velocity conversions on GI (67.3N)
#     lat2metres = 60.0*1852.0
#     lon2metres = 60.0*1852.0*np.cos(np.radians(67.3))

#     sealandratio = 2420.0/10000.0
#     sectionAreaEW = 5.73 * 1500.0 * sealandratio * lon2metres  # very approx. mean depth x length between slopes.
#     sectionAreaNS = 2.94 * 1500.0 * sealandratio * lat2metres



# model mask file
data_path = Path("data/external/iAtlantic/")
experiment_name = "VIKING20X.L46-KKG36107B"
mesh_mask_file = project_path / data_path / "mask" / experiment_name / "1_mesh_mask.nc"


## helper functions

In [ ]:
def transports(dsb, nParticles):
    transport = (sectionAreaEW*dsb.isel(obs=0).vvel*lat2metres +
                            sectionAreaNS*dsb.isel(obs=0).uvel*lon2metres)/nParticles
    speed = np.sqrt((dsb.isel(obs=0).vvel*lat2metres)**2 + (dsb.isel(obs=0).uvel*lon2metres)**2)
    return transport,speed

def normalVelocitiesT0(dsb, angle):
    return dsb.isel(obs=0).vvel*lat2metres*np.cos(angle) + dsb.isel(obs=0).uvel*lon2metres*np.sin(angle)

### read in track data

In [ ]:
ds = xr.open_dataset(project_path / path_data_tracks / filename)
ds.temp.attrs['standard_name'] = 'potential temperature'
ds.temp.attrs['units'] = 'degree_C'
ds.salt.attrs['standard_name'] = 'practical salinity'
ds.salt.attrs['units'] = 'PSU'
# dsb = xr.open_dataset(project_path / path_data_tracks / filenameb)
# dsb.temp.attrs['standard_name'] = 'temperature'
# dsb.temp.attrs['units'] = 'degree_C'

# ds_full = xr.concat([dsb,ds],dim='obs')

display(ds)


## Calculate some basic parameters from data for later

In [ ]:
nParticles = ds.sizes['traj']
nobsb = ds.sizes['obs']

# calculate initial transports

particle_transport,speed0 = transports(ds,nParticles)
normal_velocity = normalVelocitiesT0(ds,sectionAngle)
sizes = np.abs(particle_transport)/400.0


In [ ]:
display(normal_velocity*sectionArea/nParticles)
display(particle_transport)


## calculate potential densities of particles at T0

In [ ]:
# there are neater ways to do this with xarray if the datasets are large

S = ds.isel(obs=0).salt
T = ds.isel(obs=0).temp
sig0 = sw.pden(S,T,0.0,0.0)
sig0 -= 1000.0




In [ ]:
display(S)
display(T)
display(sig0)

## construct density vs. velocity transport matrix


In [ ]:
np.floor(-35.9)

In [ ]:
transportGrid = np.zeros((110,200))

x = np.arange(27.0,28.1001,0.01)
y = np.arange(-1.0,1.0001,0.01)
x1 = np.arange(27.0,28.0999,0.01)

xc=np.arange(27.005,28.100,0.01)
yc = np.arange(-0.995,1.000,0.01)


In [ ]:
for i in range(nParticles):
    transportGrid[np.floor((sig0[i]-27.0)*100.0).astype(int),np.floor((normal_velocity[i]+1.0)*100.0).astype(int)] += particle_transport[i]/1.0e06

In [ ]:
    
tGxr = xr.DataArray(data=transportGrid,coords={'sig0':xc,'velocity':yc},dims=['sig0','velocity'],name='Transport')

tGxr.attrs['units'] = r"Sv"
tGxr.sig0.attrs['units'] = r"$kg\ m^{-3}$"
tGxr.velocity.attrs['units'] = r"$m\ s^{-1}$"
   

In [ ]:
display(tGxr)

In [ ]:
transportGridSum = transportGrid.sum(axis=1)

In [ ]:
transportGrid[transportGrid==0.0] = 'NaN'

## contruct 'histogram' plot

In [ ]:
from matplotlib import rcParams
rcParams['mathtext.default'] = 'regular'
fig,ax = plt.subplots(figsize=(8,8))

tGxr.T.where(tGxr != 0.0).plot(cmap=co.cm.balance)



In [ ]:
fig,ax = plt.subplots(figsize=(8,8))

tGxrcumsum = tGxr.where(tGxr != 0.0).sum(dim='velocity').cumsum()

tGxrcumsum.attrs['units']=r"Sv"
tGxrcumsum.name=r"Cumulative Transport from low-high density"

tGxrcumsum.plot(y='sig0')

ax.set_ylim((28.1,27.0));